In [6]:
#importing packages
import pandas as pd
import math
import string
import re
import os

#getting input from the user
training_dataset_name = input("Enter the training Data you want to train. Enter either train 1 or train 2. Your answer: ")
binary_flag = input("Do you want to Binarise the document? Yes/No. Your answer: ")
stop_word_exclusion_flag = input("Do you want to exclude stop words? Yes/No. Your answer: ")

Enter the training Data you want to train. Enter either train 1 or train 2. Your answer:  train 1
Do you want to Binarise the document? Yes/No. Your answer:  no
Do you want to exclude stop words? Yes/No. Your answer:  no


In [7]:
#Initiialising training file name
training_file_name = ''

#getting path of current directory 
current_directory = os.getcwd()

#naming the filenames
if training_dataset_name.lower() == 'train 1':
    training_file_name = (current_directory + ("/training1_v1.csv"))
    
elif training_dataset_name.lower() == 'train 2':
    training_file_name = (current_directory + ("/training2_v1.csv"))
    
else :
    raise Exception("Sorry, you entered incorrect value, Run again and enter either train 1 or train 2")
       
testing_file_name = (current_directory + ("/testing1_v1.csv"))
stop_words_file_name = (current_directory + ("/StopWordList.csv"))

#reading the files
training_dataset = pd.read_csv(training_file_name)
testing_dataset = pd.read_csv(testing_file_name)
stop_words_dataset = pd.read_csv(stop_words_file_name)

#creating a list to store all stop words from the given document
stop_word_list = []
if stop_word_exclusion_flag.lower() == 'yes':
    for i in stop_words_dataset.iterrows():
        stop_word_list.append(i[1][0])

In [8]:
# function to make operations on a document
def get_document_info(iterator):
    document = iterator[1][0]
    document = document.lower()
    document_class = iterator[1][1]
    
    # Binarise the document if user input is yes
    if binary_flag.lower() == 'yes':
        lists = document.split()
        document = ''
        binarized = []
        [binarized.append(x) for x in lists if x not in binarized]
        document = ' '.join([str(elem) for elem in binarized])
        
    #exclude stop words if user input is yes
    if stop_word_exclusion_flag.lower() == 'yes':
        slists = document.split()
        document_without_stop_words = [t for t in slists if t not in stop_word_list]
        document = ''
        document = ' '.join([str(elements) for elements in document_without_stop_words])
    
    #Removing the given special characters from the document
    tokenized_document = re.split(r'[\n\t\s.,;:''""()?!]', document)
    tokenized_document = [i for i in tokenized_document if i]
        
    return document_class, tokenized_document

#method to create bag of words
def create_bag_of_words(training_dataset):
    bag_of_words_positive = []  #All words in positive (Include duplicates)
    bag_of_words_negative = []  #All words in Negative (Include duplicates)
    
    #looping through the each document in training set
    for i in training_dataset.iterrows():
        document_class, tokenized_document = get_document_info(i)
        
        if document_class.lower() == 'cancer':
            
        # loop till words are present in list tokenized_document
            for i in tokenized_document:
                
                # insert value in BagOfWordsPositive
                bag_of_words_positive.append(i) 
        
        else:
            for i in tokenized_document:
                
                # insert value in BagOfWordsNegative
                 bag_of_words_negative.append(i) 
                
    return bag_of_words_positive, bag_of_words_negative

#calling the method to create bag of words
bag_of_words_can, bag_of_words_non_can = create_bag_of_words(training_dataset)

In [9]:
#Dictionary storing Frequency of each word in positive class
freq_positive_class = {}

#Dictionary storing Frequency of each word in negative class
freq_negative_class = {} 

#removing duplicate words
unique_positive_class_words= set(bag_of_words_can)
unique_negative_class_words= set(bag_of_words_non_can)

#method to create a list of vocabulory
def create_vocabulary (uniquePositives, uniqueNegatives):
    union = set.union(uniquePositives, uniqueNegatives)
    
    return list(union)

#method to count the frequency of each word
def calculate_frequency (uniquePositives, uniqueNegatives):
    
    for x in uniquePositives:
        freq_positive_class[x] = bag_of_words_can.count(x)
        
    for y in uniqueNegatives:  
        freq_negative_class[y] = bag_of_words_non_can.count(y)
        
#Calling the functions to get vocabulary and frequency        
vocabulary = create_vocabulary(unique_positive_class_words, unique_negative_class_words)
calculate_frequency(unique_positive_class_words, unique_negative_class_words)

In [10]:
#method top calculate the perdormance matrix
def calculate_scores(testing_dataset, training_dataset):
    
    # Total number of positive and negative docs
    count_positive_doc = len(training_dataset[training_dataset['class'] == 'cancer'])
    count_negative_doc = len(training_dataset[training_dataset['class'] == 'nocancer'])

    #probability of each class
    prob_positive_class =  count_positive_doc/ (count_positive_doc + count_negative_doc)
    prob_negative_class =  count_negative_doc/ (count_positive_doc + count_negative_doc)
    
    #tp = true positive, fp = false positive, fn = false negative
    tp = 0
    fp = 0
    fn = 0

    #loop through the documents in testing dataset
    for i in testing_dataset.iterrows():
        document = i[1][0]
        document = document.lower()
        document_class = i[1][1]
        tokenized_document = re.split(r'[\n\t\s.,;:''""()?!]', document)
        tokenized_document = [i for i in tokenized_document if i]
        
        #initialising the value with the log of probability of each class calculated above
        sum_positive = math.log(prob_positive_class)
        sum_negative = math.log(prob_negative_class)
        
        #looping through the each word of test document
        for j in tokenized_document:
            
            #checking if word is present in vocabulary and if not present we will ignore the token
            if j in vocabulary:
                freq_in_positive_class = 0
                freq_in_negative_class = 0
                
                #finding the frequency of the word in positive docs
                if j in freq_positive_class.keys():
                    freq_in_positive_class = freq_positive_class[j]
                else:
                    freq_in_positive_class = 0
                
                #calculating the total probability of the word in positive docs
                probab_positive = math.log((freq_in_positive_class + 1)/ (len(vocabulary) + len(bag_of_words_can)))
                sum_positive =  probab_positive +  sum_positive
                
                #finding the frequency of the word in negative docs
                if j in freq_negative_class.keys():
                    freq_in_negative_class = freq_negative_class[j]
                else:
                    freq_in_negative_class = 0
                    
                #calculating the total probability of the word in the negative docs
                probab_negative = math.log((freq_in_negative_class + 1)/ (len(vocabulary) + len(bag_of_words_non_can)))
                sum_negative =  sum_negative +  probab_negative
                
        #comparing the values 
        if sum_positive > sum_negative:
            predicted_class = 'cancer'
            
        else:
            predicted_class = 'nocancer'
        
        # calculating true positive, false negative or false positive based on ground truth and predicted class
        if predicted_class == 'cancer' and document_class == 'cancer':
            tp = tp + 1
        
        if predicted_class == 'nocancer' and document_class == 'cancer':
            fn = fn + 1 
        
        if predicted_class == 'cancer' and document_class == 'nocancer':
            fp = fp + 1
    
    #Calculating precision, recall and f1score
    precision = tp/(tp+fp)
    recall = tp/(tp+fn)
    f1score = 2*precision*recall/ (precision+recall)
    
    # Printing the results
    print("Training set used   : " + training_dataset_name)
    print('Stop words excluded : ' + stop_word_exclusion_flag)
    print('Binary version      : ' + binary_flag)
    print('true positives      : ' + str(tp))
    print('false negatives     : ' + str(fn))
    print('false positives     : ' + str(fp))   
    print ("Precision is       : " + str(round(precision,2)))
    print ("Recall is          : " + str(round(recall,2)))
    print ("F-score is         : " + str(round(f1score,2)))

#Calling function to calculate the performance score    
calculate_scores(testing_dataset,training_dataset )

Training set used   : train 1
Stop words excluded : no
Binary version      : no
true positives      : 74
false negatives     : 26
false positives     : 36
Precision is       : 0.67
Recall is          : 0.74
F-score is         : 0.7
